In [1]:
%run 0_installs.ipynb

In [2]:
import os, uuid, time, yaml

In [3]:
#binding to kubernetes infrastructure
import pykube, json, yaml, ast
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
kubeconfs_path = ipynb_path+"/kubeconfs"
kubeapi = pykube.HTTPClient(pykube.KubeConfig.from_file(kubeconfs_path+'/k3s.yaml'))

In [4]:
# Let's create a namespace for the user
# importing base namespace specs
with open(kubeconfs_path+'/user_namespace.yaml', "r") as file:
    user_namespace = yaml.safe_load(file)
file.close()
# create namespace according to JUPYTERHUB_USER
namespace_name = os.environ['JUPYTERHUB_USER']
%store namespace_name
user_namespace['metadata']['name'] = namespace_name

#create namespace if not exists
if not (pykube.Namespace(kubeapi,user_namespace).exists()): pykube.Namespace(kubeapi,user_namespace).create()

Stored 'namespace_name' (str)


In [5]:
#importing base container specs
with open(kubeconfs_path+'/neo4j.yaml', "r") as file:
    neo4j_specs = list(yaml.safe_load_all(file))
file.close()

In [6]:
for i in range(0,len(neo4j_specs)):
    neo4j_specs[i]['metadata']['namespace'] = namespace_name

#creating deploy
if not (pykube.Deployment(kubeapi,neo4j_specs[0]).exists()): pykube.Deployment(kubeapi,neo4j_specs[0]).create()
if not (pykube.PersistentVolumeClaim(kubeapi,neo4j_specs[1]).exists()): pykube.PersistentVolumeClaim(kubeapi,neo4j_specs[1]).create()
if not (pykube.Service(kubeapi,neo4j_specs[2]).exists()): pykube.Service(kubeapi,neo4j_specs[2]).create()

In [7]:
pykube.Deployment(kubeapi,neo4j_specs[0]).exists()

True

In [8]:
#deleting deploy
# pykube.Deployment(kubeapi,neo4j_specs[0]).delete()
# pykube.PersistentVolumeClaim(kubeapi,neo4j_specs[1]).delete()
# pykube.Service(kubeapi,neo4j_specs[2]).delete()

In [9]:
#getting neo4j pod object
time.sleep(10)
for pod in pykube.Pod.objects(kubeapi).filter(namespace=namespace_name):
    if pod.metadata['name'].find('neo4j') != -1:
        neo4j_pod = pod

In [10]:
#geting pod IP if pod is ready
while not (neo4j_pod.ready):
    time.sleep(5)
    neo4j_pod.reload() #reload pod object (it changes while pod gets ready)
neo4j_pod_ip = neo4j_pod.obj['status']['podIP']

In [11]:
neo4j_pod_ip

'10.42.0.108'

In [12]:
%store neo4j_pod_ip

Stored 'neo4j_pod_ip' (str)


In [13]:
%run 2_neovis_setup.ipynb

Stored 'neo4j_vis_pod_ip' (str)


In [14]:
#pykube.Namespace(kubeapi,user_namespace).delete()